In [91]:
import exifread
# Open image file for reading (binary mode)
f = open(r"C:\Users\User\Downloads\UpdateRGB\DJI_20231015133339_0025_D.JPG", 'rb')
import json
# Return Exif tags
tags = exifread.process_file(f)
print(tags)
for t in tags:
    print(t.split()[-1])

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
fn = r"C:\Users\User\Downloads\UpdateRGB\DJI_20231015133339_0025_D.JPG"
with Image.open(fn) as img:
    # Check if image has exif data
    if hasattr(img, '_getexif'):
        exif_data = img._getexif()
        if exif_data is not None:
            # Iterate through exif data and print it
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                print(f"{tag_name}: {value}")
# import piexif
# try:
#     exif_dict = piexif.load(fn)
#     print(exif_data.keys())
#     print( xmp_data)
# except Exception as e:
#     print(f"Error reading XMP data: {e}")
    # return None
# from PIL import Image
# with Image.open(fn) as im:
#     for segment, content in im.applist:
#         marker, body = content.split(b'\x00', 1)
#         if segment == 'APP1' and marker==b'http://ns.adobe.com/xap/1.0/':
#             # parse the XML string with any method you like
#             print(body)


{'Image ImageDescription': (0x010E) ASCII=default @ 182, 'Image Make': (0x010F) ASCII=DJI @ 30, 'Image Model': (0x0110) ASCII=M3M @ 42, 'Image Orientation': (0x0112) Short=Horizontal (normal) @ 54, 'Image XResolution': (0x011A) Ratio=72 @ 190, 'Image YResolution': (0x011B) Ratio=72 @ 198, 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 90, 'Image Software': (0x0131) ASCII=10.12.05.45 @ 206, 'Image DateTime': (0x0132) ASCII=2023:10:15 13:33:39 @ 218, 'Image YCbCrPositioning': (0x0213) Short=Co-sited @ 126, 'Image XPComment': (0x9C9C) Byte=[48, 0, 46, 0, 57, 0, 46, 0, 49, 0, 52, 0, 50, 0, 0, 0, 0, 0, 0, 0, ... ] @ 238, 'Image XPKeywords': (0x9C9E) Byte=[115, 0, 105, 0, 110, 0, 103, 0, 108, 0, 101, 0] @ 366, 'Image ExifOffset': (0x8769) Long=378 @ 162, 'GPS GPSVersionID': (0x0000) Byte=[2, 3, 0, 0] @ 1038, 'GPS GPSLatitudeRef': (0x0001) ASCII=N @ 1050, 'GPS GPSLatitude': (0x0002) Ratio=20490341/826971 @ 1138, 'GPS GPSLongitudeRef': (0x0003) ASCII=E @ 1074, 'GPS GPSLongitude': (0x0004

In [87]:
import tifffile
in_folder = r"C:\Users\User\Downloads\MS_rawImages"
fn = r"C:\Users\User\Downloads\MS_rawImages\DJI_20231015133339_0025_MS_NIR.TIF"
with tifffile.TiffFile(fn) as tif:
        metadata = tif.pages[0].tags
        tif_tags = {}
        tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')
        s = open("meta.xml", 'w')
        s.write(tif_tags['XMP'])

import re
def parse_value(text, keyword):
    pattern = re.escape(keyword) + r'\s*=\s*"([^"]*)"'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

from fractions import Fraction
altitude = parse_value(tif_tags['XMP'], "drone-dji:AbsoluteAltitude")
latitude = parse_value(tif_tags['XMP'], "drone-dji:GpsLatitude")
longitude = parse_value(tif_tags['XMP'], "drone-dji:GpsLongitude")

import piexif
from PIL import Image

fn_rgb = fn[:-10] + 'D.JPG'
print(fn_rgb)
img = Image.open(fn_rgb)
exif_dict = piexif.load(img.info['exif'])
exif_dict['GPS'][piexif.GPSIFD.GPSAltitude] = (altitude.numerator, altitude.denominator)
exif_dict['GPS'][piexif.GPSIFD.GPSLongitude] = (longitude.numerator, longitude.denominator)
exif_dict['GPS'][piexif.GPSIFD.GPSLatitude] = (latitude.numerator, latitude.denominator)

exif_bytes = piexif.dump(exif_dict)
out_folder = r"C:\Users\User\Downloads\UpdateRGB"
img.save(r"C:\Users\User\Desktop\DJI_20231015133339_0025_Dn.JPG", exif=exif_bytes)

C:\Users\User\Downloads\MS_rawImages\DJI_20231015133339_0025_D.JPG


In [1]:
import os
import re
import tifffile

def parse_value(text, keyword):
    pattern = re.escape(keyword) + r'\s*=\s*"([^"]*)"'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

def process_tif(file_path, d_file_path):
    with tifffile.TiffFile(file_path) as tif:
        metadata = tif.pages[0].tags
        tif_tags = {}
        tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')

    altitude = parse_value(tif_tags['XMP'], "drone-dji:AbsoluteAltitude")
    latitude = parse_value(tif_tags['XMP'], "drone-dji:GpsLatitude")
    longitude = parse_value(tif_tags['XMP'], "drone-dji:GpsLongitude")
    pitch = parse_value(tif_tags['XMP'], "drone-dji:FlightPitchDegree")
    roll = parse_value(tif_tags['XMP'], "drone-dji:FlightRollDegree")
    yaw = parse_value(tif_tags['XMP'], "drone-dji:FlightYawDegree")
    import subprocess
    command = f'"C:/exiftool(-k).exe" -GpsLatitude="{latitude}" -GpsLongitude="{longitude}" -AbsoluteAltitude="{altitude}" -FlightPitchDegree="{pitch}" -FlightRollDegree="{roll}" -FlightYawDegree="{yaw}" "{d_file_path}"'
    subprocess.run(command, shell=True)

def process_all_tifs(input_folder, out_folder):
    for root, dirs, files in os.walk(input_folder):

        for file in files:
            try:
                if file.endswith("_MS_NIR.TIF"):
                    print(file)
                    file_path = os.path.join(root, file)
                    d_file_path = os.path.join(out_folder, file[:-10] + 'D.JPG')
                    process_tif(file_path, d_file_path)
            except:
                continue

input_folder = r"C:\Users\User\Downloads\MS_rawImages"
out_folder = r"C:\Users\User\Downloads\UpdateRGB"

process_all_tifs(input_folder, out_folder)

DJI_20231015133246_0001_MS_NIR.TIF
